# AWS Glue Studio Notebook

In [1]:
%idle_timeout 2880
%glue_version 5.0
%worker_type G.1X
%number_of_workers 5

import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
  
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.7 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 5.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 5
Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 5
Idle Timeout: 2880
Session ID: 94858b01-3f85-4f19-96f2-be95e05b0b35
Applying the following default arguments:
--glue_kernel_version 1.0.7
--enable-glue-datacatalog true
Waiting for session 94858b01-3f85-4f19-96f2-be95e05b0b35 to get into ready status...
Session 94858b01-3f85-4f19-96f2-be95e05b0b35 ha

In [12]:
from pyspark.sql.functions import *
from datetime import datetime
from awsglue.dynamicframe import DynamicFrame

In [3]:
s3_path = "s3://google-maps-etl-project-amar/raw_data/to_process/"

source_dyf = glueContext.create_dynamic_frame_from_options(
    connection_type="s3",
    format="json",
    connection_options={"paths": [s3_path]},
    format_options={"withHeader": True},
    transformation_ctx="source_dyf",
)

In [4]:
google_maps_df = source_dyf.toDF()

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:147: UserWarning: DataFrame constructor is internal. Do not directly use it.


In [5]:
google_maps_df.show()

+------+--------------------+
|status|                data|
+------+--------------------+
|    ok|[{0x3bae17259d8f9...|
+------+--------------------+


In [6]:
test_df = google_maps_df

In [7]:
test_df.withColumn("data", explode("data")).show(5, False)

+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [16]:
def process_google_maps_data(df):
    df = df.withColumn("data", explode("data"))
    df = df.select(
        col("data.name").alias("name"),
        #col("data.full_address").alias("address"),
        regexp_replace(col("data.full_address"), ",", "").alias("address"),
        col("data.review_count").alias("review_count"),
        col("data.phone_number").alias("phone_number"),
        #col("data.rating").alias("rating"),
        col("data.place_link").alias("place_link")
    )
    df = df.drop_duplicates(["name"])
    return df

In [16]:
test_df.withColumn("data", explode("data")).select(
    col("data.name").alias("name"),
    col("data.full_address").alias("address"),
    col("data.review_count").alias("review_count"),
    col("data.phone_number").alias("phone_number"),
    col("data.rating").alias("rating"),
    col("data.place_link").alias("place_link")).drop_duplicates(["name"]).show(5)

+--------------------+--------------------+------------+------------+-----------+--------------------+
|                name|             address|review_count|phone_number|     rating|          place_link|
+--------------------+--------------------+------------+------------+-----------+--------------------+
|     AB's Happy Paws|12th, AB's Happy ...|         180| 08660764838|{4.8, null}|https://www.googl...|
|          Anvis Inc.|Anvis Inc., 48, 2...|         123| 09148451003|{4.7, null}|https://www.googl...|
|Asteer pet boardi...|5th phase, Asteer...|          58| 09513808471|{4.9, null}|https://www.googl...|
|Astro Puppy Dog b...|Astro Puppy Dog b...|         103| 07892759603|{4.6, null}|https://www.googl...|
|Becky's Bow Wow H...|Becky's Bow Wow H...|         429| 08618005311|{4.6, null}|https://www.googl...|
+--------------------+--------------------+------------+------------+-----------+--------------------+
only showing top 5 rows


In [17]:
google_maps_processed_df = process_google_maps_data(test_df)
google_maps_processed_df.show(5)

+--------------------+--------------------+------------+------------+--------------------+
|                name|             address|review_count|phone_number|          place_link|
+--------------------+--------------------+------------+------------+--------------------+
|     AB's Happy Paws|12th AB's Happy P...|         180| 08660764838|https://www.googl...|
|          Anvis Inc.|Anvis Inc. 48 2nd...|         123| 09148451003|https://www.googl...|
|Asteer pet boardi...|5th phase Asteer ...|          58| 09513808471|https://www.googl...|
|Astro Puppy Dog b...|Astro Puppy Dog b...|         103| 07892759603|https://www.googl...|
|Bangalore pets villa|ground floor Bang...|          20| 07795833373|https://www.googl...|
+--------------------+--------------------+------------+------------+--------------------+
only showing top 5 rows


In [18]:
def write_to_s3(df, s3_path, format_type="csv"):
    dynamic_frame = DynamicFrame.fromDF(df, glueContext, "dynamic_frame")
    
    glueContext.write_dynamic_frame.from_options(
        frame=dynamic_frame,
        connection_type="s3",
        connection_options={"path": f"s3://google-maps-etl-project-amar/transformed_data/{s3_path}/"},
        format=format_type
    )

In [20]:
write_to_s3(google_maps_processed_df, "maps_data/transformed_data_{}".format(datetime.now().strftime("%Y-%m-%d_%H-%M-%S")), "csv")